# Figure - exp examples

In [ ]:
import os
work_dir = "H:/workspace/ptyrad"
os.chdir(work_dir)
print("Current working dir: ", os.getcwd())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2
from numpy.fft import fftshift
import h5py
from tifffile import imread, imwrite

from ptyrad.data_io import load_hdf5, load_pt

In [ ]:
def center_crop(image, crop_height, crop_width):
    """
    Center crops a 2D or 3D array (e.g., an image).

    Args:
        image (numpy.ndarray): The input array to crop. Can be 2D (H, W) or 3D (H, W, C).
        crop_height (int): The desired height of the crop.
        crop_width (int): The desired width of the crop.

    Returns:
        numpy.ndarray: The cropped image.
    """
    if len(image.shape) not in [2, 3]:
        raise ValueError("Input image must be a 2D or 3D array.")

    height, width = image.shape[-2:]

    if crop_height > height or crop_width > width:
        raise ValueError("Crop size must be smaller than the input image size.")

    start_y = (height - crop_height) // 2
    start_x = (width - crop_width) // 2

    return image[..., start_y:start_y + crop_height, start_x:start_x + crop_width]

In [ ]:
def mfft2(im):
    # Periodic Artifact Reduction in Fourier Transforms of Full Field Atomic Resolution Images
    # https://doi.org/10.1017/S1431927614014639
    rows, cols = im.shape
    
    # Compute boundary conditions
    s = np.zeros_like(im)
    s[0, :] = im[0, :] - im[rows-1, :]
    s[rows-1, :] = -s[0, :]
    s[:, 0] += im[:, 0] - im[:, cols-1]
    s[:, cols-1] -= im[:, 0] - im[:, cols-1]

    # Create grid for computing Poisson solution
    cx, cy = np.meshgrid(2 * np.pi * np.arange(cols) / cols, 
                          2 * np.pi * np.arange(rows) / rows)

    # Generate smooth component from Poisson Eq with boundary condition
    D = 2 * (2 - np.cos(cx) - np.cos(cy))
    D[0, 0] = np.inf  # Enforce zero mean & handle division by zero
    S = np.fft.fft2(s) / D

    P = np.fft.fft2(im) - S  # FFT of periodic component
    return P, S

In [ ]:
# Define model data as a dictionary

reconstructions = {
    "MOSS6": {
        "title": 'MOSS-6',
        "path": "H:/workspace/ptyrad/output/paper/MOSS6/20250210_full_N65536_dp128_flipT100_random512_p2_1obj_5slice_dz40_Adam_plr1e-5_oplr5e-4_dpblur1_orblur0.5_ozblur1_opos_sng1.0_spr0.1/model_iter0020.pt",
        "dose": 100,
        "crop_param": [384, 384] # This is for final real space display only, the FFT is calculated with nearly the full size image for better k-space sampling
    },
    "ZSM5": {
        "title": 'ZSM-5',
        "path": "H:/workspace/ptyrad/output/paper/ZSM5/20250204_full_N16384_dp128_flipT100_random32_p6_1obj_8slice_dz40_Adam_ps10_ss50_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_ozblur1_oathr0.98_opos_sng1.0_spr0.1_tilt-1.2_4.7/model_iter2000.pt",
        "dose": 3500,
        "crop_param": [142, 142]
    },
    "tBL_WSe2": {
        "title": 'tBL-WSe2',
        "path": "H:/workspace/ptyrad/output/paper/tBL_WSe2/20250131_ptyrad_convergence/full_N16384_dp128_flipT100_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.5_ozblur1_oathr0.98_opos_sng1.0_spr0.1_aff1_0_-3_0/model_iter4000.pt",
        "dose": 7.55e5,
        "crop_param": [192, 192]
    },
    "PSO": {
        "title": 'PrScO$_3$',
        "path": "H:/workspace/ptyrad/output/paper/PSO/20241202_full_N4096_dp128_sparse32_p8_1obj_21slice_dz10_Adam_ps20_ss50_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_kzf1_opos_sng1.0_spr0.1_aff1_0.05_-0.5_-4_t0562_aff_1_0.05_-0.5_-4/model_iter1000.pt",
        "dose": 1.22e6,
        "crop_param": [136, 136]
    }
}

# Load reconstructions and add relevant entries back to the dict
for name, data in reconstructions.items():
    
    print(f"\nCurrently processing: {name}")
    # Load model checkpoints
    ckpt = load_pt(data["path"])

    # Get dx_size and thickness from ckpt
    dx_size   = ckpt['model_attributes']['dx'].cpu().numpy()[()]
    thickness = ckpt['model_attributes']['slice_thickness'] * ckpt['optimizable_tensors']['objp'].shape[1]/10
    data["dx_size"] = dx_size
    data["thickness"] = thickness
    print(f"Loaded dx_size = {dx_size:.4f} Ang, thickness = {thickness:.2g} nm")
    
    # Extract and store object
    obj = ckpt["optimizable_tensors"]["objp"].detach().squeeze().cpu().numpy().sum(0)
    data["obj"] = obj
    print(f"Loaded obj.shape = {obj.shape}, obj.dtype = {obj.dtype}")

    # Compute FFT and dk size
    crop_side = min(obj.shape)
    fft = np.log(np.abs(fftshift(mfft2(center_crop(obj, crop_side, crop_side))[0]))) # FFT is calculated on nearly the full size obj for finer k-space sampling
    dk = 1 / (dx_size * crop_side)
    data["crop_side"] = crop_side
    data["fft"] = fft
    data["dk_size"] = dk
    print(f'Crop the object as square with side length = {crop_side} px before FFT, so the FFT would have finer k-sampling due to larger N (comparing to the displayed obj range)')
    print(f'fft.shape={fft.shape}, fft.dtype={fft.dtype}, dk = 1/(N*dx) = {dk:.6f} 1/Ang')

    # Get crop ranges
    # # Back-up code if we don't know how much to crop
    # crop_pos = ckpt['model_attributes']['crop_pos'].cpu().numpy()
    # ymin, ymax, xmin, xmax = crop_pos[:, 0].min(), crop_pos[:, 0].max(), crop_pos[:, 1].min(), crop_pos[:, 1].max()
    # crop_height, crop_width = ymax - ymin, xmax - xmin
    
    crop_height, crop_width = data['crop_param']
    crop_obj = center_crop(obj, crop_height, crop_width)
    data["crop_obj"] = crop_obj
    print(f'Crop the obj image into reasonable display range with crop_height = {crop_height}, crop_width = {crop_width}]') # Note that this cropping has no effect on the FFT or image scale bar
    print(f'crop_obj.shape = {crop_obj.shape}, dtype={crop_obj.dtype}')
    
    # Save everything back into reconstructions dict
    reconstructions[name] = data



In [ ]:
# Iterate through reconstructions dynamically
for i, (name, data) in enumerate(reconstructions.items()):
    if i >= 4:
        break  # Only process the first 4 reconstructions for top row

    title = data['title']
    obj = data['crop_obj']
    dx_size = data['dx_size']
    dose = data['dose']
    thickness = data['thickness']
    fft = data['fft']
    dk_size = data['dk_size']
    
    imwrite(f'fft_{name}_dk{dk_size:.5f}.tif', np.float32(fft))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patheffects as path_effects

# Create figure with GridSpec
fig = plt.figure(figsize=(7, 3.5), dpi=300)
gs = GridSpec(2, 4, figure=fig, wspace=0.04, hspace=0.015)  # Control spacing

# Titles for the columns
panel_labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
shadow_offset = (0.01, -0.01)  # Relative unit

# Iterate through reconstructions dynamically
for i, (name, data) in enumerate(reconstructions.items()):
    if i >= 4:
        break  # Only process the first 4 reconstructions for top row
    
    ax = fig.add_subplot(gs[0, i])
    label = panel_labels[i]

    title = data['title']
    obj = data['crop_obj']
    dx_size = data['dx_size']
    dose = data['dose']
    thickness = data['thickness']
    fft = data['fft']
    dk_size = data['dk_size']
    
    # Scale bar settings
    scale_bar_length = (20 if i == 0 else 10) / dx_size
    scale_bar_label = "2 nm" if i == 0 else "1 nm"
    fontprops = fm.FontProperties(size=10)
    
    # Main image
    vmin, vmax = np.percentile(obj, [1, 99.95])
    ax.imshow(obj, cmap='gray', origin='upper', vmin=vmin, vmax=vmax)
    ax.set_title(title, fontsize=12)
    ax.axis('off')
    
    # Panel label with shadow
    ax.text(5 + shadow_offset[0], 5 + shadow_offset[1], label, color='black', fontsize=16, fontweight='bold', va='top', ha='left', alpha=0.6)
    ax.text(5, 5, label, color='white', fontsize=16, fontweight='bold', va='top', ha='left')
    
    # Dose label formatting
    if i < 2:
        dose_label = f'${dose}\,e^{{-}}/\mathrm{{\AA}}^{{2}}$'
    else:
        coef, exp = f"{dose:.1e}".split("e")
        dose_label = f"${coef} \\times 10^{{{int(exp)}}}\,e^{{-}}/\\mathrm{{\\AA}}^{{2}}$"
    thickness_label = f"${thickness:.2g}\,\mathrm{{nm}}$"
    
    # Define path effects for outlined text
    outline_effect = [path_effects.Stroke(linewidth=1.5, foreground='black'), path_effects.Normal()]
    
    # Place dose and thickness labels
    text = ax.text(0.95, 0.95, dose_label, color='white', fontsize=9, fontweight='bold', va='top', ha='right', transform=ax.transAxes)
    text.set_path_effects(outline_effect)
    text = ax.text(0.95, 0.82, thickness_label, color='white', fontsize=9, fontweight='bold', va='top', ha='right', transform=ax.transAxes)
    text.set_path_effects(outline_effect)
    
    # Scale bar
    scalebar = AnchoredSizeBar(ax.transData, scale_bar_length, scale_bar_label,
                               loc='lower right', pad=0.5, color="white", frameon=False,
                               size_vertical=obj.shape[0] * 0.01, label_top=True, fontproperties=fontprops)
    ax.add_artist(scalebar)
    
    # FFT Processing (Bottom row)
    ax_fft = fig.add_subplot(gs[1, i])
    
    # Scale bar settings for FFT
    if i == 0:
        scale_bar_length = 0.4 / dk_size
        scale_bar_label = "4 nm$^{-1}$"
        fft = center_crop(fft, 440, 440)  # Crop the center part of fft for display. This doesn't change the dk pixel size
    elif i == 1:
        scale_bar_length = 1 / dk_size
        scale_bar_label = "1 $\\mathrm{\AA}^{-1}$"
    else:
        scale_bar_length = 2 / dk_size
        scale_bar_label = "2 $\\mathrm{\AA}^{-1}$"
    
    # Main FFT image
    if i == 0:
        vmin, vmax = np.percentile(fft, [20, 99.95])
    else:
        vmin, vmax = np.percentile(fft, [10, 99.95])
    ax_fft.imshow(fft, cmap='magma', vmin=vmin, vmax=vmax)
    ax_fft.axis('off')
    
    # Panel label with shadow
    ax_fft.text(5 + shadow_offset[0], 5 + shadow_offset[1], panel_labels[i + 4], color='black', fontsize=16, fontweight='bold', va='top', ha='left', alpha=0.6)
    ax_fft.text(5, 5, panel_labels[i + 4], color='white', fontsize=16, fontweight='bold', va='top', ha='left')
    
    # Scale bar for FFT
    scalebar = AnchoredSizeBar(ax_fft.transData, scale_bar_length, scale_bar_label,
                               loc='lower right', pad=0.5, color="white", frameon=False,
                               size_vertical=fft.shape[0] * 0.01, label_top=True, fontproperties=fontprops)
    ax_fft.add_artist(scalebar)

# Show the plot
plt.show()
